In [ ]:
from google.colab import drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Diplomski/

/content/drive/MyDrive/Diplomski


In [ ]:
import torch
import cv2
from matplotlib import pyplot as plt
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

Train/Test/Val

In [ ]:
#Read images and annotations, split into train-test

images=[os.path.join('/content/drive/MyDrive/Diplomski/yolov5/data/Moxa3K/images',x) for x in os.listdir('/content/drive/MyDrive/Diplomski/yolov5/data/Moxa3K/images')]
annotations=[os.path.join('/content/drive/MyDrive/Diplomski/yolov5/data/Moxa3K/annotations',x) for x in os.listdir('/content/drive/MyDrive/Diplomski/yolov5/data/Moxa3K/annotations')]
images.sort()
annotations.sort()
train_images, test20_images, train_annotations, test20_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)


In [ ]:
# Split the dataset into train-valid-test splits
train_images, test20_images, train_annotations, test20_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)


In [ ]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)

# Move the splits into their folders
move_files_to_folder(train_images, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/train')
move_files_to_folder(test20_images, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/test20/images')
move_files_to_folder(train_annotations, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/train')
move_files_to_folder(test20_annotations, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/test20/annotations')


In [ ]:
# Split test20 into test and val
test_images, val_images, test_annotations, val_annotations = train_test_split(test20_images, test20_annotations, test_size = 0.5, random_state = 1)
move_files_to_folder(test_images, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/test')
move_files_to_folder(val_images, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/val')
move_files_to_folder(test_annotations, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/test')
move_files_to_folder(val_annotations, '/content/drive/MyDrive/Diplomski/yolov5/data/Data/val')


Statistika

In [ ]:
d = {}

for folder in ['test']:
  for file in os.listdir(folder):
    if '.jpg' in file:
      picture_name = file[:-4]
      image = PIL.Image.open(os.getcwd()+f'/{folder}/'+picture_name+'.jpg')
      image = cv2.imread(os.getcwd()+f'/{folder}/'+picture_name+'.jpg')
      height, width, _ = image.shape
      d[picture_name] = {}
      d[picture_name]['width'] = int(width)
      d[picture_name]['height'] = int(height)
      d[picture_name]['area'] = height*width

      with open(os.getcwd()+f'/{folder}/'+picture_name+'.txt', 'r') as f:
        lbl_list = []
        for line in f.readlines():
          lbl_data = line.split()
          lbl_dict = {}
          lbl_dict['class'] = lbl_data[0]
          lbl_dict['x'] = float(lbl_data[1]) * width # x,y, width i height su
          lbl_dict['y'] = float(lbl_data[2]) * height # proporcije u odnosu na sliku
          lbl_dict['width'] = float(lbl_data[3]) * width # pa množimo sa
          lbl_dict['height'] = float(lbl_data[4]) * height # dimenzijama slike
          lbl_dict['area'] = lbl_dict['width'] * lbl_dict['height']

          lbl_list.append(lbl_dict)
        d[picture_name]['labels'] = lbl_list
        if len(d) % 10 == 0:
          print(len(d))

In [ ]:
os.getcwd()

'/content/drive/MyDrive/Diplomski/Data'

In [ ]:
img_num = 3000
d = {}

for i in range(img_num):
  picture_name = 'MOXA_'+str(i)
  image=PIL.Image.open(os.getcwd()+f'/{folder}/'+picture_name+'.jpg')
  width, height = image.size
  d[picture_name] = {}
  d[picture_name]['width'] = int(width)
  d[picture_name]['height'] = int(height)
  d[picture_name]['area'] = height*width
  with open(os.getcwd()+f'/{folder}/'+picture_name+'.txt','r') as f:
    lbl_list = []
    for line in f.readlines():
      lbl_data = line.split()
      lbl_dict = {}
      lbl_dict['class'] = lbl_data[0]
      lbl_dict['x'] = float(lbl_data[1]) * width # x,y, width i height su
      lbl_dict['y'] = float(lbl_data[2]) * height # proporcije u odnosu na sliku
      lbl_dict['width'] = float(lbl_data[3]) * width # pa množimo sa
      lbl_dict['height'] = float(lbl_data[4]) * height # dimenzijama slike
      lbl_dict['area'] = lbl_dict['width'] * lbl_dict['height']

      lbl_list.append(lbl_dict)
    d[picture_name]['labels'] = lbl_list



In [ ]:
l = []
for key, value in d.items():
  for label in value['labels']:
    label['picture'] = key
    label['picture_width'] = value['width']
    label['picture_height'] = value['height']
    label['picture_area'] = value['area']
    l.append(label)
df = pd.DataFrame(l)



In [ ]:
df.describe() # Osnovni statisticki podaci

YOLOv5


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
^C


In [ ]:
%cd yolov5

/content/drive/MyDrive/Diplomski/yolov5


In [ ]:
!pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.0/617.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-217-g8c45e51 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [ ]:
!pip install clearml

In [ ]:
!clearml-init

Face mask detection

In [ ]:
!python train.py --img 640 --batch 16 --epochs 20 --data MOXA_colab.yaml --weights yolov5l.pt --cache --save-period 5

In [ ]:
!python train.py --img 640 --batch 16 --epochs 40 --data MOXA_colab.yaml --weights yolov5l.pt --cache --save-period 5 --optimizer Adam

Testing

In [ ]:
!python val.py --weights runs/train/exp/weights/epoch5.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp/weights/epoch10.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp/weights/epoch15.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp/weights/last.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch5.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch10.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch15.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch20.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch25.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch30.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/epoch35.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

In [ ]:
!python val.py --weights runs/train/exp3/weights/last.pt --data MOXA_colab.yaml --img 640 --task test --batch-size 1

FACE DETECTION

In [ ]:
import os
import time
import uuid
import cv2
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import albumentations as alb

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=640, height=640),
                         alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2),
                         alb.RGBShift(p=0.2),
                         alb.VerticalFlip(p=0.5)],
                       bbox_params=alb.BboxParams(format='albumentations',
                                                  label_fields=['class_labels']))

In [ ]:
#funkcija za ucitavanje slika
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
for partition in ['train','test','val']:
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001] #difoltni set koordinata
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [1280,720,1280,720]))

        try:
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0:
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0
                    else:
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else:
                    annotation['bbox'] = [0,0,0,0] #ovo je deo za ako ne postoji klasa
                    annotation['class'] = 0
                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

In [ ]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
final = tf.data.Dataset.list_files('aug_data\\final\\images\\*.jpg', shuffle=False)
final = final.map(load_image)
final = final.map(lambda x: tf.image.resize(x, (120,120)))
final = final.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
testMasks_images = tf.data.Dataset.list_files('aug_data\\testMasks\\images\\*.jpg', shuffle=False)
testMasks_images = testMasks_images.map(load_image)
testMasks_images = testMasks_images.map(lambda x: tf.image.resize(x, (120,120)))
testMasks_images = testMasks_images.map(lambda x: x/255)

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)

    return [label['class']], label['bbox'] #vracamo klasu i bounding box koorinate, class je 0 ili 1

In [ ]:
final_labels = tf.data.Dataset.list_files('aug_data\\final\\labels\\*.json', shuffle=False)
final_labels = final_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
testMasks_labels = tf.data.Dataset.list_files('aug_data\\testMasks\\labels\\*.json', shuffle=False)
testMasks_labels = testMasks_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16])

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
testMask = tf.data.Dataset.zip((testMasks_images, testMasks_labels))
testMask = testMask.shuffle(1300)
testMask = testMask.batch(8)
testMask = testMask.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

VGG16

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
def build_model():
    input_layer = Input(shape=(120,120,3))
    #naseg inputa

    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(1, activation='sigmoid')(class1)


    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)

    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [ ]:
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=lr_decay
)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule) #adam optimizer

In [ ]:
def localization_loss(y_true, yhat):
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))

    h_true = y_true[:,3] - y_true[:,1]
    w_true = y_true[:,2] - y_true[:,0] #actual width of the box

    h_pred = yhat[:,3] - yhat[:,1] #predicted high of the box
    w_pred = yhat[:,2] - yhat[:,0] #predicted with of the box

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))

    return delta_coord + delta_size

Traing

In [ ]:
class FaceTracker(Model):

    def __init__(self, eyetracker,  **kwargs):
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def train_step(self, batch, **kwargs):

        X, y = batch

        with tf.GradientTape() as tape:
            classes, coords = self.model(X, training=True)

            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)

            total_loss = batch_localizationloss+0.5*batch_classloss

            grad = tape.gradient(total_loss, self.model.trainable_variables)

        opt.apply_gradients(zip(grad, self.model.trainable_variables))

        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}


    def test_step(self, batch, **kwargs):
        X, y = batch
        classes, coords = self.model(X, training=False)

        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss + 0.5 * batch_classloss

        # Calculate classification predictions
        #predicted_labels = tf.argmax(classes, axis=-1)
       # true_labels = tf.argmax(y[0], axis=-1)
        predicted_labels = classes
        true_labels = y[0]
        threshold = 0.5
        binary_predicted_labels = (predicted_labels > threshold).astype(int)

        # Calculate metrics
        precision = precision_score(true_labels, binary_predicted_labels, average='weighted')
        recall = recall_score(true_labels, binary_predicted_labels, average='weighted')
        f1 = f1_score(true_labels, binary_predicted_labels, average='weighted')

        # Calculate IoU and Average Precision
        predicted_bboxes = coords  # Assuming coords are bounding box predictions
        true_bboxes = y[1]  # Assuming y[1] contains true bounding boxes
        iou = calculate_iou(true_bboxes, predicted_bboxes)  # Modify to calculate IoU
        ap = average_precision_score(true_labels, binary_predicted_labels, average='weighted')  # Calculate AP

        return {
            "total_loss":total_loss,
            "class_loss":batch_classloss,
            "regress_loss":batch_localizationloss,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "iou": iou,
            "ap": ap
        }
    def call(self, X, **kwargs):
        return self.model(X, **kwargs)


In [ ]:
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
precision_scores = []
recall_scores = []
f1_scores = []
iou_scores = []
ap_scores = []

# Iterate through test set and evaluate the model
for epoch in range(10):
    # ... (Code for model training here if needed)

    # Initialize metrics for this epoch
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    total_iou = 0
    total_ap = 0

    for batch in test:
        images, labels = batch
        predictions = model.predict(images)  # Assuming 'model' is your FaceTracker model

        # Calculate IoU
        #iou_metric = tf.keras.metrics.MeanIoU(num_classes=1)  # Set the number of classes

        #labels_argmax = tf.argmax(labels, axis=-1, output_type=tf.uint8)
        #predictions_argmax = tf.argmax(predictions, axis=-1, output_type=tf.uint8)

        #iou_metric.update_state(labels_argmax, predictions_argmax)

        #iou_value = iou_metric.result().numpy()
        #total_iou += iou_value

        # Flatten the predictions and labels for binary classification
        flat_predictions = np.reshape(predictions[0], (-1,))
        flat_labels = np.reshape(labels[0], (-1,))

        # Calculate precision, recall, and F1 score
        precision = precision_score(flat_labels, (flat_predictions > 0.9).astype(int))
        recall = recall_score(flat_labels, (flat_predictions > 0.9).astype(int))
        f1 = f1_score(flat_labels, (flat_predictions > 0.9).astype(int))

        total_precision += precision
        total_recall += recall
        total_f1 += f1
        #total_iou += iou

        # Calculate Average Precision
        ap = average_precision_score(flat_labels, flat_predictions)
        total_ap += ap

    # Calculate averages for this epoch
    avg_precision = total_precision / len(test)
    avg_recall = total_recall / len(test)
    avg_f1 = total_f1 / len(test)
    #avg_iou = total_iou / len(test)
    avg_ap = total_ap / len(test)

    # Append values to lists
    precision_scores.append(avg_precision)
    recall_scores.append(avg_recall)
    f1_scores.append(avg_f1)
    #iou_scores.append(avg_iou)
    ap_scores.append(avg_ap)

    # Print metrics for this epoch
    print(f"Epoch {epoch + 1}:")
    print(f"Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}, F1: {avg_f1:.4f}")
   # print(f"IoU: {avg_iou:.4f}, Avg Precision: {avg_ap:.4f}")
    print()

# Visualize metrics
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), precision_scores, label='Precision')
plt.plot(range(1, 11), recall_scores, label='Recall')
plt.plot(range(1, 11), f1_scores, label='F1')
plt.plot(range(1, 11), iou_scores, label='IoU')
plt.plot(range(1, 11), ap_scores, label='Avg Precision')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Evaluation Metrics Over 10 Epochs')
plt.legend()
plt.grid()
plt.show()